# Trabajo Práctico 1
## Métricas custom para reducción de falsos positivos en clasificación binaria fraude

**Mark Albrand - 21004**

### Función a optimizar

*Detectar fraudes en regiones con alta incidencia histórica*
